# VAE notebook

In [1]:
import torch
import torchvision
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os

In [2]:
if not os.path.exists('./vae_img'):
    os.mkdir('./vae_img')


def to_img(x):
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


## Main Variables

In [3]:
num_epochs = 100
batch_size = 128
learning_rate = 1e-3


## Build Model

In [4]:
img_transform = transforms.Compose([
    transforms.ToTensor()
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = MNIST('./data', transform=img_transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [5]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return F.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar

In [6]:
model = VAE()
if torch.cuda.is_available():
    model.cuda()

reconstruction_function = nn.MSELoss(size_average=False)

/usr/local/lib/python3.6/site-packages/torch/nn/functional.py:54: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [7]:
def loss_function(recon_x, x, mu, logvar):
    """
    recon_x: generating images
    x: origin images
    mu: latent mean
    logvar: latent log variance
    """
    BCE = reconstruction_function(recon_x, x)  # mse loss
    # loss = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)
    # KL divergence
    return BCE + KLD

In [8]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)

## Train Model

In [10]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(dataloader):
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img)
        if torch.cuda.is_available():
            img = img.cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(img)
        loss = loss_function(recon_batch, img, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch,
                batch_idx * len(img),
                len(dataloader.dataset), 100. * batch_idx / len(dataloader),
                loss.item() / len(img)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(dataloader.dataset)))
    if epoch % 10 == 0:
        save = to_img(recon_batch.cpu().data)
        save_image(save, './vae_img/image_{}.png'.format(epoch))

/usr/local/lib/python3.6/site-packages/torch/nn/functional.py:1101: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train Epoch: 0 [0/60000 (0%)]	Loss: 185.856583
Train Epoch: 0 [12800/60000 (21%)]	Loss: 48.857880
Train Epoch: 0 [25600/60000 (43%)]	Loss: 41.187515
Train Epoch: 0 [38400/60000 (64%)]	Loss: 40.885494
Train Epoch: 0 [51200/60000 (85%)]	Loss: 37.236664
====> Epoch: 0 Average loss: 45.3193
Train Epoch: 1 [0/60000 (0%)]	Loss: 35.896774
Train Epoch: 1 [12800/60000 (21%)]	Loss: 35.381142
Train Epoch: 1 [25600/60000 (43%)]	Loss: 34.825508
Train Epoch: 1 [38400/60000 (64%)]	Loss: 35.963440
Train Epoch: 1 [51200/60000 (85%)]	Loss: 34.907967
====> Epoch: 1 Average loss: 34.9878
Train Epoch: 2 [0/60000 (0%)]	Loss: 33.119278
Train Epoch: 2 [12800/60000 (21%)]	Loss: 33.468979
Train Epoch: 2 [25600/60000 (43%)]	Loss: 33.750629
Train Epoch: 2 [38400/60000 (64%)]	Loss: 31.996033
Train Epoch: 2 [51200/60000 (85%)]	Loss: 32.026268
====> Epoch: 2 Average loss: 33.1248
Train Epoch: 3 [0/60000 (0%)]	Loss: 32.645252
Train Epoch: 3 [12800/60000 (21%)]	Loss: 33.107098
Train Epoch: 3 [25600/60000 (43%)]	Loss: 

In [11]:
torch.save(model.state_dict(), './vae.pth')